In [5]:
from pymongo import MongoClient

client= MongoClient("mongodb://localhost:27017/")
db= client.companies


In [6]:
import pandas as pd
df= pd.DataFrame(db.comp.find())
df.head()

,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297d8b,None,[],Zoho ManageEngine,,,enterprise,[],2007-05-25 19:24:22,http://www.crunchbase.com/company/adventnet,...,925-924-9500,[],"[{'title': 'DHFH', 'is_past': True, 'provider'...","[{'is_past': True, 'title': 'CEO and Co-Founde...","[{'available_sizes': [[[150, 94], 'assets/imag...",,$0,manageengine,Wed Oct 31 18:26:09 UTC 2012,[]
1,52cdef7c4bab8bd675297d8a,"{'price_amount': 30000000, 'price_currency_cod...",[],,http://digitalquarters.net/feed/,http://digitalquarters.net/,web,"[{'competitor': {'name': 'Wikia', 'permalink':...",2007-05-25 06:51:27,http://www.crunchbase.com/company/wetpaint,...,206.859.6300,"[{'name': 'Wikison Wetpaint', 'permalink': 'we...",[],"[{'is_past': False, 'title': 'Co-Founder and V...","[{'available_sizes': [[[150, 86], 'assets/imag...","wiki, seattle, elowitz, media-industry, media-...",$39.8M,BachelrWetpaint,Sun Dec 08 07:15:44 UTC 2013,[]
2,52cdef7c4bab8bd675297d8c,None,[],,http://blogs.zoho.com/feed,http://blogs.zoho.com/,software,"[{'competitor': {'name': 'Empressr', 'permalin...",Fri May 25 19:30:28 UTC 2007,http://www.crunchbase.com/company/zoho,...,1-888-204-3539,"[{'name': 'Zoho Office Suite', 'permalink': 'z...",[],"[{'is_past': False, 'title': 'CEO and Founder'...",[],"zoho, officesuite, spreadsheet, writer, projec...",$0,zoho,Wed Oct 30 00:07:05 UTC 2013,"[{'embed_code': '<object width=""430"" height=""2..."
3,52cdef7c4bab8bd675297d8d,"{'price_amount': 500000, 'price_currency_code'...","[{'price_amount': None, 'price_currency_code':...",,http://blog.digg.com/?feed=rss2,http://blog.digg.com/,news,"[{'competitor': {'name': 'Reddit', 'permalink'...",Fri May 25 20:03:23 UTC 2007,http://www.crunchbase.com/company/digg,...,(415) 436-9638,"[{'name': 'Digg', 'permalink': 'digg'}]","[{'title': 'Public Relations', 'is_past': True...","[{'is_past': False, 'title': 'CEO', 'person': ...","[{'available_sizes': [[[117, 150], 'assets/ima...","community, social, news, bookmark, digg, techn...",$45M,digg,Tue Nov 05 21:35:47 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl..."
4,52cdef7c4bab8bd675297d8f,None,[],None,http://feeds.feedburner.com/omnidrive,http://www.omnidrive.com/blog,network_hosting,"[{'competitor': {'name': 'Dropbox', 'permalink...",Sun May 27 03:25:32 UTC 2007,http://www.crunchbase.com/company/omnidrive,...,660-675-5052,"[{'name': 'Omnidrive', 'permalink': 'omnidrive'}]",[],"[{'is_past': True, 'title': 'Co-founder', 'per...",[],"storage, sharing, edit, online",$800k,Nomadesk,Tue Jul 02 22:48:04 UTC 2013,[]


In [7]:
df.columns

Index(['_id', 'acquisition', 'acquisitions', 'alias_list', 'blog_feed_url',
       'blog_url', 'category_code', 'competitions', 'created_at',
       'crunchbase_url', 'deadpooled_day', 'deadpooled_month',
       'deadpooled_url', 'deadpooled_year', 'description', 'email_address',
       'external_links', 'founded_day', 'founded_month', 'founded_year',
       'funding_rounds', 'homepage_url', 'image', 'investments', 'ipo',
       'milestones', 'name', 'number_of_employees', 'offices', 'overview',
       'partners', 'permalink', 'phone_number', 'products', 'providerships',
       'relationships', 'screenshots', 'tag_list', 'total_money_raised',
       'twitter_username', 'updated_at', 'video_embeds'],
      dtype='object')

In [8]:
at_least_1_office = db.comp.find({
    "offices":{
        "$not":{
            "$size":0
        }
    }
},{"name":1, "offices":1})


In [9]:
df_at_least_1_office = pd.DataFrame(at_least_1_office)
df_at_least_1_office.head()

,_id,name,offices
0,52cdef7c4bab8bd675297d8b,AdventNet,"[{'description': 'Headquarters', 'address1': '..."
1,52cdef7c4bab8bd675297d8a,Wetpaint,"[{'description': '', 'address1': '710 - 2nd Av..."
2,52cdef7c4bab8bd675297d8c,Zoho,"[{'description': 'Headquarters', 'address1': '..."
3,52cdef7c4bab8bd675297d8d,Digg,"[{'description': None, 'address1': '135 Missis..."
4,52cdef7c4bab8bd675297d8f,Omnidrive,"[{'description': '', 'address1': 'Suite 200', ..."


In [10]:
def getFirst(data):
    data = data['offices']
    principal = None
    if data[0]['latitude'] and data[0]['longitude']:
        principal = {
            "type":"Point",
            "coordinates":[data[0]['longitude'], data[0]['latitude']]
        }

    return {
        "totalOffices": len(data),
        "lat": data[0]['latitude'],
        "lng": data[0]['longitude'],
        "oficina_principal": principal
    }


first_office = df_at_least_1_office[["offices"]].apply(getFirst, result_type="expand", axis=1)

In [11]:
df_clean = pd.concat([df_at_least_1_office,first_office], axis=1)[["name","lat","lng", "oficina_principal","totalOffices"]]

In [12]:
display(df_clean.shape)
df_clean.head()

(13744, 5)

,name,lat,lng,oficina_principal,totalOffices
0,AdventNet,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,...",1.0
1,Wetpaint,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,...",2.0
2,Zoho,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,...",1.0
3,Digg,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,...",1.0
4,Omnidrive,NaN,NaN,NaN,1.0


In [13]:
df["description"].value_counts().head()

                                 2990
software solutions                 17
Search Engine Optimization         14
fabless semiconductor company      11
Social Network                     11
Name: description, dtype: int64

In [14]:
#!pip3 install textdistance
#!pip3 install textdistance[Hamming]

In [15]:
df["total_money_raised"].value_counts().head()

$0      13976
$1M        93
$10M       90
$5M        84
$2M        79
Name: total_money_raised, dtype: int64

In [16]:
#limpio la columna "total_money_raised"
import re
def clean(x):
    if "M" or "m" in str(x) and "€" in str(x):
        non_decimal = re.compile(r'[^\d.]+')
        a=float(non_decimal.sub('', str(x)))*1000000*1.12
        return a
    if "K" or "k" in str(x) and "€" in str(x):
        non_decimal = re.compile(r'[^\d.]+')
        a=float(non_decimal.sub('', str(x)))*1000*1.12
        return a
    if "M" or "m" in str(x) and "$" in str(x):
        non_decimal = re.compile(r'[^\d.]+')
        a=float(non_decimal.sub('', str(x)))*1000000
        return a
    if "K" or "k" in str(x) and "$" in str(x):
        non_decimal = re.compile(r'[^\d.]+')
        a=float(non_decimal.sub('', str(x)))*1000
        return a
    if "M" or "m" in str(x) and "£" in str(x):
        non_decimal = re.compile(r'[^\d.]+')
        a=float(non_decimal.sub('', str(x)))*1000000*1.25
        return a
    if "K" or "k" in str(x) and "£" in str(x):
        non_decimal = re.compile(r'[^\d.]+')
        a=float(non_decimal.sub('', str(x)))*1000*1.25
        return a
    return None   
df2=df.copy()
df2["total_money_raised"]=df2["total_money_raised"].apply(clean)
df2.head()

,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297d8b,None,[],Zoho ManageEngine,,,enterprise,[],2007-05-25 19:24:22,http://www.crunchbase.com/company/adventnet,...,925-924-9500,[],"[{'title': 'DHFH', 'is_past': True, 'provider'...","[{'is_past': True, 'title': 'CEO and Co-Founde...","[{'available_sizes': [[[150, 94], 'assets/imag...",,0.0,manageengine,Wed Oct 31 18:26:09 UTC 2012,[]
1,52cdef7c4bab8bd675297d8a,"{'price_amount': 30000000, 'price_currency_cod...",[],,http://digitalquarters.net/feed/,http://digitalquarters.net/,web,"[{'competitor': {'name': 'Wikia', 'permalink':...",2007-05-25 06:51:27,http://www.crunchbase.com/company/wetpaint,...,206.859.6300,"[{'name': 'Wikison Wetpaint', 'permalink': 'we...",[],"[{'is_past': False, 'title': 'Co-Founder and V...","[{'available_sizes': [[[150, 86], 'assets/imag...","wiki, seattle, elowitz, media-industry, media-...",44576000.0,BachelrWetpaint,Sun Dec 08 07:15:44 UTC 2013,[]
2,52cdef7c4bab8bd675297d8c,None,[],,http://blogs.zoho.com/feed,http://blogs.zoho.com/,software,"[{'competitor': {'name': 'Empressr', 'permalin...",Fri May 25 19:30:28 UTC 2007,http://www.crunchbase.com/company/zoho,...,1-888-204-3539,"[{'name': 'Zoho Office Suite', 'permalink': 'z...",[],"[{'is_past': False, 'title': 'CEO and Founder'...",[],"zoho, officesuite, spreadsheet, writer, projec...",0.0,zoho,Wed Oct 30 00:07:05 UTC 2013,"[{'embed_code': '<object width=""430"" height=""2..."
3,52cdef7c4bab8bd675297d8d,"{'price_amount': 500000, 'price_currency_code'...","[{'price_amount': None, 'price_currency_code':...",,http://blog.digg.com/?feed=rss2,http://blog.digg.com/,news,"[{'competitor': {'name': 'Reddit', 'permalink'...",Fri May 25 20:03:23 UTC 2007,http://www.crunchbase.com/company/digg,...,(415) 436-9638,"[{'name': 'Digg', 'permalink': 'digg'}]","[{'title': 'Public Relations', 'is_past': True...","[{'is_past': False, 'title': 'CEO', 'person': ...","[{'available_sizes': [[[117, 150], 'assets/ima...","community, social, news, bookmark, digg, techn...",50400000.0,digg,Tue Nov 05 21:35:47 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl..."
4,52cdef7c4bab8bd675297d8f,None,[],None,http://feeds.feedburner.com/omnidrive,http://www.omnidrive.com/blog,network_hosting,"[{'competitor': {'name': 'Dropbox', 'permalink...",Sun May 27 03:25:32 UTC 2007,http://www.crunchbase.com/company/omnidrive,...,660-675-5052,"[{'name': 'Omnidrive', 'permalink': 'omnidrive'}]",[],"[{'is_past': True, 'title': 'Co-founder', 'per...",[],"storage, sharing, edit, online",896000000.0,Nomadesk,Tue Jul 02 22:48:04 UTC 2013,[]


In [17]:
#calculo la textdistance y la utilizo para filtrar mi dataframe por medio de la columna "description"

import textdistance
df2["description"]=df2["description"].dropna()
#display(df['description'])
df_filtered= pd.DataFrame()
for i in df2["description"]:
    words = [
        ["marketing",str(i)],
        ["design",str(i)],
        ["advertising",str(i)],
        ["web",str(i)]
    ]


    metric = textdistance.hamming.normalized_distance
    zipped = zip(words,map(lambda x :metric(x[0],x[1]), words))

    for words, score in zipped:
        if score<0.75:
            #print(words,score)
            df_filtered=df_filtered.append(df2[df2['description'] == i]) 
df_filtered["description"].value_counts().head()
df_filtered.head()

,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
26,52cdef7c4bab8bd675297da5,None,"[{'price_amount': None, 'price_currency_code':...",,,http://technoratimedia.com/tm-blog/,advertising,"[{'competitor': {'name': 'Twingly', 'permalink...",Tue Jun 12 04:06:06 UTC 2007,http://www.crunchbase.com/company/technorati,...,415-896-3000,"[{'name': 'Technorati', 'permalink': 'technora...",[],"[{'is_past': False, 'title': 'Chairman', 'pers...",[],"blogs, search",35952000.0,technorati,Tue Nov 12 07:23:57 UTC 2013,"[{'embed_code': 'Technorati <div style=""width:..."
355,52cdef7c4bab8bd675297eeb,None,[],,http://blog.zopa.com/feed/,http://blog.zopa.com,finance,"[{'competitor': {'name': 'Lending Club', 'perm...",Thu Aug 23 09:23:31 UTC 2007,http://www.crunchbase.com/company/zopa,...,0207 580 6060,"[{'name': 'Zopa', 'permalink': 'zopa'}]",[],"[{'is_past': False, 'title': 'Co-Founder and C...",[],prosper,37968000.0,zopa,Wed Sep 11 03:35:00 UTC 2013,[]
433,52cdef7c4bab8bd675297f33,"{'price_amount': 1300000000, 'price_currency_c...","[{'price_amount': 200000000, 'price_currency_c...",,,,web,[],Fri Aug 24 12:13:41 UTC 2007,http://www.crunchbase.com/company/digitas,...,,[],[],"[{'is_past': False, 'title': 'Vice President |...",[],None,0.0,,Sat Jun 22 02:40:50 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl..."
564,52cdef7c4bab8bd675297fbd,"{'price_amount': None, 'price_currency_code': ...",[],,http://blog.oneriot.com/feed/,http://blog.oneriot.com/,advertising,[],Mon Sep 03 14:17:10 UTC 2007,http://www.crunchbase.com/company/oneriot,...,303-938-3040,[{'name': 'OneRiot For Developers & Publishers...,[],"[{'is_past': False, 'title': 'CEO', 'person': ...","[{'available_sizes': [[[87, 150], 'assets/imag...","realtime, real-time, ad-network, realtime-web,...",30576000.0,OneRiot,Wed Sep 14 02:33:21 UTC 2011,"[{'embed_code': '<div style=""width:320px;""><di..."
700,52cdef7c4bab8bd675298046,"{'price_amount': None, 'price_currency_code': ...",[],,,http://adready.com/site/category/blog/,advertising,"[{'competitor': {'name': 'AdKiwi', 'permalink'...",Tue Oct 16 10:46:12 UTC 2007,http://www.crunchbase.com/company/adready,...,(888) 897-7225,[],[],"[{'is_past': False, 'title': 'Chairman of the ...","[{'available_sizes': [[[150, 95], 'assets/imag...","advertising, banner, display",19936000.0,AdReady,Mon Aug 12 13:09:26 UTC 2013,[]


In [18]:
df_final=df_clean.merge(df_filtered, on="name")

In [19]:
df_final.head()
df_final_filtered=df_final[["name","lat","lng","oficina_principal","totalOffices","description","total_money_raised"]]
df_definitivo=df_final_filtered[df_final_filtered["total_money_raised"]>1000000]

In [20]:
df_definitivo_sinduplicados=df_definitivo.drop_duplicates(subset="name", keep='first', inplace=False)
df_definitivo_sinduplicados.head()
df_sinna=df_definitivo_sinduplicados.dropna()

In [21]:
#!pip3 install python-google-places

In [23]:
#utilizo la API de googleplaces para obtener el número de restaurantes que se encuentran alrededor de cada empresa.
#calculo la media de las valoraciones
#radio empleado: 500 metros

import os
from dotenv import load_dotenv
load_dotenv()
from googleplaces import GooglePlaces, types, lang

API_KEY=os.environ["api"]

df_sinna["lat"]=df_sinna["lat"].replace(",",".",regex=True)
df_sinna["lng"]=df_sinna["lng"].replace(",",".",regex=True)

google_places = GooglePlaces(API_KEY)
def find(x):
    query_result = google_places.nearby_search(
            lat_lng={'lat': x["lat"], 'lng': x["lng"]}, 
            radius=500,
            types=[types.TYPE_RESTAURANT] or [types.TYPE_CAFE] or [type.TYPE_BAR] or [type.TYPE_CASINO])

    if query_result.has_attributions:
        print(query_result.html_attributions)

    
    lst=[]
    for place in query_result.places:
        place.get_details()
        lst.append(re.sub("[A-Za-z]","",str(place.rating)))
    suma=0
    num=0
    for i in lst:
        try:
            suma+=float(i)
            num+=1
    
        except ValueError:
            pass
    try:
        media=suma/num
    except ZeroDivisionError:
        media=None
    return {
        "RestaurantsRating": media,
        "RestaurantsNumber":num
            
    }
        

df_api=df_sinna[["lat","lng"]].apply(find,result_type="expand",axis=1)
df_api.head()

/home/luis/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/luis/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,RestaurantsNumber,RestaurantsRating
0,19.0,4.215789
1,20.0,4.165000
3,20.0,4.350000
4,20.0,4.355000
6,20.0,4.180000


In [24]:
merged= df_sinna.merge(df_api, left_index=True, right_index=True, how='inner')

In [25]:
merged.shape

(27, 9)

In [26]:
definitivo=merged.dropna()

In [27]:
definitivo.head()

,name,lat,lng,oficina_principal,totalOffices,description,total_money_raised,RestaurantsNumber,RestaurantsRating
0,Technorati,37.779558,-122.393041,"{'type': 'Point', 'coordinates': [-122.393041,...",1.0,Advertising platform/Publisher network,35952000.0,19.0,4.215789
1,Zopa,51.517904,-0.139947,"{'type': 'Point', 'coordinates': [-0.139947, 5...",1.0,P2P Lending,37968000.0,20.0,4.165000
3,OneRiot,40.016210,-105.281310,"{'type': 'Point', 'coordinates': [-105.28131, ...",1.0,Advertising Network for Twitter Apps,30576000.0,20.0,4.350000
4,AdReady,47.649027,-122.346299,"{'type': 'Point', 'coordinates': [-122.346299,...",1.0,Advertising technology company,19936000.0,20.0,4.355000
6,Predictify,37.486571,-122.229161,"{'type': 'Point', 'coordinates': [-122.229161,...",1.0,Marketing Prediction Software,4816000.0,20.0,4.180000


In [28]:
#obtengo el número de restaurantes en un radio de 1000 metros

from googleplaces import GooglePlaces, types, lang

API_KEY=os.environ["api"]

google_places = GooglePlaces(API_KEY)
def find(x):
    query_result = google_places.nearby_search(
            lat_lng={'lat': x["lat"], 'lng': x["lng"]}, 
            radius=1000,
            types=[types.TYPE_SUBWAY_STATION] or [types.TYPE_TAXI_STAND] or [type.TYPE_BUS_STATION] or [type.TYPE_PARKING])

    if query_result.has_attributions:
        print(query_result.html_attributions)

    
    i=1
    for place in query_result.places:
        i=i+1
    return {
        "TravelNumber":i
            
    }
        

df_api2=df_sinna[["lat","lng"]].apply(find,result_type="expand",axis=1)
df_api2.head()

,TravelNumber
0,1
1,9
3,1
4,1
6,1


In [29]:
merged1= definitivo.merge(df_api2, left_index=True, right_index=True, how='inner')

In [30]:
merged1.head()

,name,lat,lng,oficina_principal,totalOffices,description,total_money_raised,RestaurantsNumber,RestaurantsRating,TravelNumber
0,Technorati,37.779558,-122.393041,"{'type': 'Point', 'coordinates': [-122.393041,...",1.0,Advertising platform/Publisher network,35952000.0,19.0,4.215789,1
1,Zopa,51.517904,-0.139947,"{'type': 'Point', 'coordinates': [-0.139947, 5...",1.0,P2P Lending,37968000.0,20.0,4.165000,9
3,OneRiot,40.016210,-105.281310,"{'type': 'Point', 'coordinates': [-105.28131, ...",1.0,Advertising Network for Twitter Apps,30576000.0,20.0,4.350000,1
4,AdReady,47.649027,-122.346299,"{'type': 'Point', 'coordinates': [-122.346299,...",1.0,Advertising technology company,19936000.0,20.0,4.355000,1
6,Predictify,37.486571,-122.229161,"{'type': 'Point', 'coordinates': [-122.229161,...",1.0,Marketing Prediction Software,4816000.0,20.0,4.180000,1


In [31]:
#obtengo el número de gimnasios y estadios

from googleplaces import GooglePlaces, types, lang

API_KEY=os.environ["api"]

google_places = GooglePlaces(API_KEY)
def find(x):
    query_result = google_places.nearby_search(
            lat_lng={'lat': x["lat"], 'lng': x["lng"]}, 
            radius=1000,
            types=[types.TYPE_GYM] or [types.TYPE_STADIUM])

    if query_result.has_attributions:
        print(query_result.html_attributions)

    
    i=1
    for place in query_result.places:
        i=i+1
    return {
        "SportNumber":i
            
    }
        

df_api3=df_sinna[["lat","lng"]].apply(find,result_type="expand",axis=1)
df_api3.head()

,SportNumber
0,21
1,21
3,10
4,21
6,17


In [32]:
merged2= merged1.merge(df_api3, left_index=True, right_index=True, how='inner')
merged2.head()

,name,lat,lng,oficina_principal,totalOffices,description,total_money_raised,RestaurantsNumber,RestaurantsRating,TravelNumber,SportNumber
0,Technorati,37.779558,-122.393041,"{'type': 'Point', 'coordinates': [-122.393041,...",1.0,Advertising platform/Publisher network,35952000.0,19.0,4.215789,1,21
1,Zopa,51.517904,-0.139947,"{'type': 'Point', 'coordinates': [-0.139947, 5...",1.0,P2P Lending,37968000.0,20.0,4.165000,9,21
3,OneRiot,40.016210,-105.281310,"{'type': 'Point', 'coordinates': [-105.28131, ...",1.0,Advertising Network for Twitter Apps,30576000.0,20.0,4.350000,1,10
4,AdReady,47.649027,-122.346299,"{'type': 'Point', 'coordinates': [-122.346299,...",1.0,Advertising technology company,19936000.0,20.0,4.355000,1,21
6,Predictify,37.486571,-122.229161,"{'type': 'Point', 'coordinates': [-122.229161,...",1.0,Marketing Prediction Software,4816000.0,20.0,4.180000,1,17


In [33]:
#obtengo el número de aeropuertos o estaciones de tren en un radio de 10km

from googleplaces import GooglePlaces, types, lang

API_KEY=os.environ["api"]
google_places = GooglePlaces(API_KEY)
def find(x):
    query_result = google_places.nearby_search(
            lat_lng={'lat': x["lat"], 'lng': x["lng"]}, 
            radius=10000,
            types=[types.TYPE_TRAIN_STATION] or [types.TYPE_AIRPORT])

    if query_result.has_attributions:
        print(query_result.html_attributions)

    
    i=1
    for place in query_result.places:
        i=i+1
    return {
        "TRAIN_AIRPORT_Number":i
            
    }
        

df_api4=df_sinna[["lat","lng"]].apply(find,result_type="expand",axis=1)
df_api4.head()

,TRAIN_AIRPORT_Number
0,6
1,21
3,1
4,3
6,9


In [34]:
merged3= merged2.merge(df_api4, left_index=True, right_index=True, how='inner')
merged3.head()

,name,lat,lng,oficina_principal,totalOffices,description,total_money_raised,RestaurantsNumber,RestaurantsRating,TravelNumber,SportNumber,TRAIN_AIRPORT_Number
0,Technorati,37.779558,-122.393041,"{'type': 'Point', 'coordinates': [-122.393041,...",1.0,Advertising platform/Publisher network,35952000.0,19.0,4.215789,1,21,6
1,Zopa,51.517904,-0.139947,"{'type': 'Point', 'coordinates': [-0.139947, 5...",1.0,P2P Lending,37968000.0,20.0,4.165000,9,21,21
3,OneRiot,40.016210,-105.281310,"{'type': 'Point', 'coordinates': [-105.28131, ...",1.0,Advertising Network for Twitter Apps,30576000.0,20.0,4.350000,1,10,1
4,AdReady,47.649027,-122.346299,"{'type': 'Point', 'coordinates': [-122.346299,...",1.0,Advertising technology company,19936000.0,20.0,4.355000,1,21,3
6,Predictify,37.486571,-122.229161,"{'type': 'Point', 'coordinates': [-122.229161,...",1.0,Marketing Prediction Software,4816000.0,20.0,4.180000,1,17,9


In [35]:
#obtengo el número de sitios de copas en un radio de 1000 metros

from googleplaces import GooglePlaces, types, lang

API_KEY=os.environ["api"]
google_places = GooglePlaces(API_KEY)
def find(x):
    query_result = google_places.nearby_search(
            lat_lng={'lat': x["lat"], 'lng': x["lng"]}, 
            radius=1000,
            types=[types.TYPE_NIGHT_CLUB])

    if query_result.has_attributions:
        print(query_result.html_attributions)

    
    i=1
    for place in query_result.places:
        i=i+1
    return {
        "Night_club_number":i
            
    }
        

df_api5=df_sinna[["lat","lng"]].apply(find,result_type="expand",axis=1)
df_api5.head()

,Night_club_number
0,21
1,21
3,10
4,4
6,7


In [36]:
merged4= merged3.merge(df_api5, left_index=True, right_index=True, how='inner')
merged4.head()

,name,lat,lng,oficina_principal,totalOffices,description,total_money_raised,RestaurantsNumber,RestaurantsRating,TravelNumber,SportNumber,TRAIN_AIRPORT_Number,Night_club_number
0,Technorati,37.779558,-122.393041,"{'type': 'Point', 'coordinates': [-122.393041,...",1.0,Advertising platform/Publisher network,35952000.0,19.0,4.215789,1,21,6,21
1,Zopa,51.517904,-0.139947,"{'type': 'Point', 'coordinates': [-0.139947, 5...",1.0,P2P Lending,37968000.0,20.0,4.165000,9,21,21,21
3,OneRiot,40.016210,-105.281310,"{'type': 'Point', 'coordinates': [-105.28131, ...",1.0,Advertising Network for Twitter Apps,30576000.0,20.0,4.350000,1,10,1,10
4,AdReady,47.649027,-122.346299,"{'type': 'Point', 'coordinates': [-122.346299,...",1.0,Advertising technology company,19936000.0,20.0,4.355000,1,21,3,4
6,Predictify,37.486571,-122.229161,"{'type': 'Point', 'coordinates': [-122.229161,...",1.0,Marketing Prediction Software,4816000.0,20.0,4.180000,1,17,9,7


In [37]:
#obtengo el número de colegios en un radio de 1000 metros
#El 30% de mis empleados tienen hijos y el hecho de tener un colegio cerca al que llevar a sus hijos les facilitaría su día a día

from googleplaces import GooglePlaces, types, lang

API_KEY=os.environ["api"]
google_places = GooglePlaces(API_KEY)
def find(x):
    query_result = google_places.nearby_search(
            lat_lng={'lat': x["lat"], 'lng': x["lng"]}, 
            radius=1000,
            types=[types.TYPE_SCHOOL])

    if query_result.has_attributions:
        print(query_result.html_attributions)

    
    i=1
    for place in query_result.places:
        i=i+1
    return {
        "Number_SCHOOL":i
            
    }
        

df_api6=df_sinna[["lat","lng"]].apply(find,result_type="expand",axis=1)
df_api6.head()

,Number_SCHOOL
0,21
1,21
3,21
4,21
6,21


In [38]:
merged5= merged4.merge(df_api6, left_index=True, right_index=True, how='inner')
merged5.head()

,name,lat,lng,oficina_principal,totalOffices,description,total_money_raised,RestaurantsNumber,RestaurantsRating,TravelNumber,SportNumber,TRAIN_AIRPORT_Number,Night_club_number,Number_SCHOOL
0,Technorati,37.779558,-122.393041,"{'type': 'Point', 'coordinates': [-122.393041,...",1.0,Advertising platform/Publisher network,35952000.0,19.0,4.215789,1,21,6,21,21
1,Zopa,51.517904,-0.139947,"{'type': 'Point', 'coordinates': [-0.139947, 5...",1.0,P2P Lending,37968000.0,20.0,4.165000,9,21,21,21,21
3,OneRiot,40.016210,-105.281310,"{'type': 'Point', 'coordinates': [-105.28131, ...",1.0,Advertising Network for Twitter Apps,30576000.0,20.0,4.350000,1,10,1,10,21
4,AdReady,47.649027,-122.346299,"{'type': 'Point', 'coordinates': [-122.346299,...",1.0,Advertising technology company,19936000.0,20.0,4.355000,1,21,3,4,21
6,Predictify,37.486571,-122.229161,"{'type': 'Point', 'coordinates': [-122.229161,...",1.0,Marketing Prediction Software,4816000.0,20.0,4.180000,1,17,9,7,21


In [39]:
df_total=merged5.loc[:,["RestaurantsNumber","RestaurantsRating","TravelNumber","SportNumber","TRAIN_AIRPORT_Number","Night_club_number","Number_SCHOOL"]]

In [40]:
df_total.head()

,RestaurantsNumber,RestaurantsRating,TravelNumber,SportNumber,TRAIN_AIRPORT_Number,Night_club_number,Number_SCHOOL
0,19.0,4.215789,1,21,6,21,21
1,20.0,4.165000,9,21,21,21,21
3,20.0,4.350000,1,10,1,10,21
4,20.0,4.355000,1,21,3,4,21
6,20.0,4.180000,1,17,9,7,21


In [41]:
normalized_df=(df_total-df_total.mean())/df_total.std()

In [42]:
normalized_df.head()

,RestaurantsNumber,RestaurantsRating,TravelNumber,SportNumber,TRAIN_AIRPORT_Number,Night_club_number,Number_SCHOOL
0,0.533564,-0.332930,-0.475876,0.767100,-0.104127,1.193557,0.382873
1,0.680754,-0.646535,2.211425,0.767100,2.573418,1.193557,0.382873
3,0.680754,0.495766,-0.475876,-0.879358,-0.996642,-0.056836,0.382873
4,0.680754,0.526639,-0.475876,0.767100,-0.639636,-0.738869,0.382873
6,0.680754,-0.553916,-0.475876,0.168388,0.431382,-0.397852,0.382873


In [43]:
#creo un input que me permite asignar pesos a las columnas para ponderarlas.

con=input("Quieres ponderar el peso de las columnas (y/n)?: ")
if con=="y":
    print("\n")
    print(normalized_df.columns)
    print("\nHay {0} columnas".format(len(normalized_df.columns)))
    print("\nRecuerda: la suma de los pesos debe ser 1")
    pesos=[]
    for i in range(len(normalized_df.columns)):
        a=float(input("\n{0}) Introduce el peso que quieras asignarle a la columna {1}:".format(i+1,normalized_df.columns[i])))
        pesos.append(a)
    if sum(pesos)==1:
        for i in range(len(pesos)):
            normalized_df[normalized_df.columns[i]] *= pesos[i]
    else:
        print("Error. La suma de los pesos asignados debe ser 1. Sin embargo los números que ha introducido suman {0}".format(sum(pesos)))

Quieres ponderar el peso de las columnas (y/n)?: 


In [44]:
a=normalized_df.sum(axis = 1, skipna = True)

In [45]:
normalized_df['SUM'] = a

In [46]:
normalized_df.head()

,RestaurantsNumber,RestaurantsRating,TravelNumber,SportNumber,TRAIN_AIRPORT_Number,Night_club_number,Number_SCHOOL,SUM
0,0.533564,-0.332930,-0.475876,0.767100,-0.104127,1.193557,0.382873,1.964161
1,0.680754,-0.646535,2.211425,0.767100,2.573418,1.193557,0.382873,7.162592
3,0.680754,0.495766,-0.475876,-0.879358,-0.996642,-0.056836,0.382873,-0.849319
4,0.680754,0.526639,-0.475876,0.767100,-0.639636,-0.738869,0.382873,0.502985
6,0.680754,-0.553916,-0.475876,0.168388,0.431382,-0.397852,0.382873,0.235753


In [47]:
df_primera_parte=merged5.loc[:,["name","lat","lng","oficina_principal","totalOffices","description","total_money_raised"]]

In [48]:
final=df_primera_parte.merge(normalized_df, left_index=True, right_index=True, how='inner')

In [49]:
final.head()

,name,lat,lng,oficina_principal,totalOffices,description,total_money_raised,RestaurantsNumber,RestaurantsRating,TravelNumber,SportNumber,TRAIN_AIRPORT_Number,Night_club_number,Number_SCHOOL,SUM
0,Technorati,37.779558,-122.393041,"{'type': 'Point', 'coordinates': [-122.393041,...",1.0,Advertising platform/Publisher network,35952000.0,0.533564,-0.332930,-0.475876,0.767100,-0.104127,1.193557,0.382873,1.964161
1,Zopa,51.517904,-0.139947,"{'type': 'Point', 'coordinates': [-0.139947, 5...",1.0,P2P Lending,37968000.0,0.680754,-0.646535,2.211425,0.767100,2.573418,1.193557,0.382873,7.162592
3,OneRiot,40.016210,-105.281310,"{'type': 'Point', 'coordinates': [-105.28131, ...",1.0,Advertising Network for Twitter Apps,30576000.0,0.680754,0.495766,-0.475876,-0.879358,-0.996642,-0.056836,0.382873,-0.849319
4,AdReady,47.649027,-122.346299,"{'type': 'Point', 'coordinates': [-122.346299,...",1.0,Advertising technology company,19936000.0,0.680754,0.526639,-0.475876,0.767100,-0.639636,-0.738869,0.382873,0.502985
6,Predictify,37.486571,-122.229161,"{'type': 'Point', 'coordinates': [-122.229161,...",1.0,Marketing Prediction Software,4816000.0,0.680754,-0.553916,-0.475876,0.168388,0.431382,-0.397852,0.382873,0.235753


In [50]:
z=final[['SUM']].idxmax() 

In [51]:
print(final["name"][z],final["lat"][z],final["lng"][z])

30    [x+1]
Name: name, dtype: object 30    40.745507
Name: lat, dtype: float64 30   -73.982624
Name: lng, dtype: float64


In [52]:
at_least_1_office = db.comp.find({
    "offices":{
        "$not":{
            "$size":0
        }
    }
},{"name":1, "offices":1,"founded_year":1})


In [53]:
df_at_least_1_office = pd.DataFrame(at_least_1_office)
df_at_least_1_office.head()

,_id,founded_year,name,offices
0,52cdef7c4bab8bd675297d8b,1996.0,AdventNet,"[{'description': 'Headquarters', 'address1': '..."
1,52cdef7c4bab8bd675297d8a,2005.0,Wetpaint,"[{'description': '', 'address1': '710 - 2nd Av..."
2,52cdef7c4bab8bd675297d8c,2005.0,Zoho,"[{'description': 'Headquarters', 'address1': '..."
3,52cdef7c4bab8bd675297d8d,2004.0,Digg,"[{'description': None, 'address1': '135 Missis..."
4,52cdef7c4bab8bd675297d8f,2005.0,Omnidrive,"[{'description': '', 'address1': 'Suite 200', ..."


In [54]:
def getFirst(data):
    data = data['offices']
    principal = None
    if data[0]['latitude'] and data[0]['longitude']:
        principal = {
            "type":"Point",
            "coordinates":[data[0]['longitude'], data[0]['latitude']]
        }

    return {
        "totalOffices": len(data),
        "lat": data[0]['latitude'],
        "lng": data[0]['longitude'],
        "oficina_principal": principal
    }


first_office = df_at_least_1_office[["offices"]].apply(getFirst, result_type="expand", axis=1)

In [55]:
df_clean = pd.concat([df_at_least_1_office,first_office], axis=1)[["name","lat","lng", "oficina_principal","totalOffices","founded_year"]]

In [56]:
df_clean1=df_clean.drop_duplicates(subset="name", keep='first', inplace=False)

In [57]:
df_clean1=df_clean1[df_clean1["founded_year"]<2009]

In [58]:
df_clean1.dropna().head()

,name,lat,lng,oficina_principal,totalOffices,founded_year
0,AdventNet,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,...",1.0,1996.0
1,Wetpaint,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,...",2.0,2005.0
2,Zoho,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,...",1.0,2005.0
3,Digg,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,...",1.0,2004.0
5,Postini,37.506885,-122.247573,"{'type': 'Point', 'coordinates': [-122.247573,...",1.0,1999.0


In [59]:
#calculo el número de empresas de más de 10 años que se encuentran en un radio inferior a 2km.

import math
lat_list = final["lat"].tolist()
lng_list = final["lng"].tolist()
names_list= final["name"].tolist()
lst=[]
for i in range(len(lat_list)):
    def distance(x):
        radius = 6371 # km
        dlat = math.radians(x["lat"]-lat_list[i])
        dlon = math.radians(x["lng"]-(lng_list[i]))
        a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat_list[i])) \
            * math.cos(math.radians(x["lat"])) * math.sin(dlon/2) * math.sin(dlon/2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        d = radius * c
        return d


    df_distance=df_clean1[["lat","lng","name"]].apply(distance,result_type="expand",axis=1)
    num=0
    for e in df_distance:
        if e<2 and e!=0:
            num=num+1
    lst.append(num)

In [60]:
lst

[270,
 50,
 15,
 18,
 29,
 260,
 3,
 0,
 265,
 0,
 0,
 54,
 311,
 41,
 21,
 250,
 252,
 33,
 0,
 341,
 0,
 1,
 0,
 3]

In [61]:
empresas_cercanas = pd.DataFrame({'NUM_empresas_cercanas':lst})
print (empresas_cercanas)

    NUM_empresas_cercanas
0                     270
1                      50
2                      15
3                      18
4                      29
5                     260
6                       3
7                       0
8                     265
9                       0
10                      0
11                     54
12                    311
13                     41
14                     21
15                    250
16                    252
17                     33
18                      0
19                    341
20                      0
21                      1
22                      0
23                      3


In [62]:
fin=final.reset_index(drop=True)

In [63]:
res = pd.concat([fin, empresas_cercanas], axis=1)

In [64]:
res.head()

,name,lat,lng,oficina_principal,totalOffices,description,total_money_raised,RestaurantsNumber,RestaurantsRating,TravelNumber,SportNumber,TRAIN_AIRPORT_Number,Night_club_number,Number_SCHOOL,SUM,NUM_empresas_cercanas
0,Technorati,37.779558,-122.393041,"{'type': 'Point', 'coordinates': [-122.393041,...",1.0,Advertising platform/Publisher network,35952000.0,0.533564,-0.332930,-0.475876,0.767100,-0.104127,1.193557,0.382873,1.964161,270
1,Zopa,51.517904,-0.139947,"{'type': 'Point', 'coordinates': [-0.139947, 5...",1.0,P2P Lending,37968000.0,0.680754,-0.646535,2.211425,0.767100,2.573418,1.193557,0.382873,7.162592,50
2,OneRiot,40.016210,-105.281310,"{'type': 'Point', 'coordinates': [-105.28131, ...",1.0,Advertising Network for Twitter Apps,30576000.0,0.680754,0.495766,-0.475876,-0.879358,-0.996642,-0.056836,0.382873,-0.849319,15
3,AdReady,47.649027,-122.346299,"{'type': 'Point', 'coordinates': [-122.346299,...",1.0,Advertising technology company,19936000.0,0.680754,0.526639,-0.475876,0.767100,-0.639636,-0.738869,0.382873,0.502985,18
4,Predictify,37.486571,-122.229161,"{'type': 'Point', 'coordinates': [-122.229161,...",1.0,Marketing Prediction Software,4816000.0,0.680754,-0.553916,-0.475876,0.168388,0.431382,-0.397852,0.382873,0.235753,29


In [65]:
x=(res["NUM_empresas_cercanas"]-res["NUM_empresas_cercanas"].mean())/res["NUM_empresas_cercanas"].std()

del res['NUM_empresas_cercanas']

In [66]:
res1 = pd.concat([res, x], axis=1)
res1.head()

,name,lat,lng,oficina_principal,totalOffices,description,total_money_raised,RestaurantsNumber,RestaurantsRating,TravelNumber,SportNumber,TRAIN_AIRPORT_Number,Night_club_number,Number_SCHOOL,SUM,NUM_empresas_cercanas
0,Technorati,37.779558,-122.393041,"{'type': 'Point', 'coordinates': [-122.393041,...",1.0,Advertising platform/Publisher network,35952000.0,0.533564,-0.332930,-0.475876,0.767100,-0.104127,1.193557,0.382873,1.964161,1.429786
1,Zopa,51.517904,-0.139947,"{'type': 'Point', 'coordinates': [-0.139947, 5...",1.0,P2P Lending,37968000.0,0.680754,-0.646535,2.211425,0.767100,2.573418,1.193557,0.382873,7.162592,-0.341096
2,OneRiot,40.016210,-105.281310,"{'type': 'Point', 'coordinates': [-105.28131, ...",1.0,Advertising Network for Twitter Apps,30576000.0,0.680754,0.495766,-0.475876,-0.879358,-0.996642,-0.056836,0.382873,-0.849319,-0.622827
3,AdReady,47.649027,-122.346299,"{'type': 'Point', 'coordinates': [-122.346299,...",1.0,Advertising technology company,19936000.0,0.680754,0.526639,-0.475876,0.767100,-0.639636,-0.738869,0.382873,0.502985,-0.598679
4,Predictify,37.486571,-122.229161,"{'type': 'Point', 'coordinates': [-122.229161,...",1.0,Marketing Prediction Software,4816000.0,0.680754,-0.553916,-0.475876,0.168388,0.431382,-0.397852,0.382873,0.235753,-0.510135


In [69]:
con=input("Quieres ponderar el peso de las columnas (y/n)?: ")
if con=="y":
    print("\n")
    print(res1.columns[-2]," " , res1.columns[-1])
    print("\nRecuerda: la suma de los pesos debe ser 1")
    pesos=[]
    for i in range(2):
        a=float(input("\n{0}) Introduce el peso que quieras asignarle a la columna {1}:".format(i+1,res.columns[-(i+1)])))
        pesos.append(a)
    if sum(pesos)==1:
        for i in range(len(pesos)):
            res1[res1.columns[-(i+1)]] *= pesos[i]
    else:
        print("Error. La suma de los pesos asignados debe ser 1. Sin embargo los números que ha introducido suman {0}".format(sum(pesos)))

Quieres ponderar el peso de las columnas (y/n)?: n


In [70]:
res1["SUM_TOTAL"] = res1["SUM"] - res1["NUM_empresas_cercanas"]

In [71]:
res1.head()

,name,lat,lng,oficina_principal,totalOffices,description,total_money_raised,RestaurantsNumber,RestaurantsRating,TravelNumber,SportNumber,TRAIN_AIRPORT_Number,Night_club_number,Number_SCHOOL,SUM,NUM_empresas_cercanas,SUM_TOTAL
0,Technorati,37.779558,-122.393041,"{'type': 'Point', 'coordinates': [-122.393041,...",1.0,Advertising platform/Publisher network,35952000.0,0.533564,-0.332930,-0.475876,0.767100,-0.104127,1.193557,0.382873,1.964161,1.429786,0.534374
1,Zopa,51.517904,-0.139947,"{'type': 'Point', 'coordinates': [-0.139947, 5...",1.0,P2P Lending,37968000.0,0.680754,-0.646535,2.211425,0.767100,2.573418,1.193557,0.382873,7.162592,-0.341096,7.503688
2,OneRiot,40.016210,-105.281310,"{'type': 'Point', 'coordinates': [-105.28131, ...",1.0,Advertising Network for Twitter Apps,30576000.0,0.680754,0.495766,-0.475876,-0.879358,-0.996642,-0.056836,0.382873,-0.849319,-0.622827,-0.226492
3,AdReady,47.649027,-122.346299,"{'type': 'Point', 'coordinates': [-122.346299,...",1.0,Advertising technology company,19936000.0,0.680754,0.526639,-0.475876,0.767100,-0.639636,-0.738869,0.382873,0.502985,-0.598679,1.101664
4,Predictify,37.486571,-122.229161,"{'type': 'Point', 'coordinates': [-122.229161,...",1.0,Marketing Prediction Software,4816000.0,0.680754,-0.553916,-0.475876,0.168388,0.431382,-0.397852,0.382873,0.235753,-0.510135,0.745888


In [72]:
empresas_cercanas1 = pd.DataFrame({'NUM_empresas':lst})

In [73]:
res2 = pd.concat([res1, empresas_cercanas1], axis=1)
res2.head()

,name,lat,lng,oficina_principal,totalOffices,description,total_money_raised,RestaurantsNumber,RestaurantsRating,TravelNumber,SportNumber,TRAIN_AIRPORT_Number,Night_club_number,Number_SCHOOL,SUM,NUM_empresas_cercanas,SUM_TOTAL,NUM_empresas
0,Technorati,37.779558,-122.393041,"{'type': 'Point', 'coordinates': [-122.393041,...",1.0,Advertising platform/Publisher network,35952000.0,0.533564,-0.332930,-0.475876,0.767100,-0.104127,1.193557,0.382873,1.964161,1.429786,0.534374,270
1,Zopa,51.517904,-0.139947,"{'type': 'Point', 'coordinates': [-0.139947, 5...",1.0,P2P Lending,37968000.0,0.680754,-0.646535,2.211425,0.767100,2.573418,1.193557,0.382873,7.162592,-0.341096,7.503688,50
2,OneRiot,40.016210,-105.281310,"{'type': 'Point', 'coordinates': [-105.28131, ...",1.0,Advertising Network for Twitter Apps,30576000.0,0.680754,0.495766,-0.475876,-0.879358,-0.996642,-0.056836,0.382873,-0.849319,-0.622827,-0.226492,15
3,AdReady,47.649027,-122.346299,"{'type': 'Point', 'coordinates': [-122.346299,...",1.0,Advertising technology company,19936000.0,0.680754,0.526639,-0.475876,0.767100,-0.639636,-0.738869,0.382873,0.502985,-0.598679,1.101664,18
4,Predictify,37.486571,-122.229161,"{'type': 'Point', 'coordinates': [-122.229161,...",1.0,Marketing Prediction Software,4816000.0,0.680754,-0.553916,-0.475876,0.168388,0.431382,-0.397852,0.382873,0.235753,-0.510135,0.745888,29


In [74]:
z=res2[['SUM_TOTAL']].idxmax()
print(res2["name"][z],res2["lat"][z],res2["lng"][z])

1    Zopa
Name: name, dtype: object 1    51.517904
Name: lat, dtype: float64 1   -0.139947
Name: lng, dtype: float64


In [75]:
res2.to_csv('output/dataset_final.csv')